In [ ]:
import json
import time
from datetime import datetime
from pathlib import Path
from typing import Optional

class ResultLogger:
    def __init__(self, results_directory: str = "results"):
        self.results_dir = Path(results_directory)
        self.results_dir.mkdir(exist_ok=True)
        
    def save_result(self, member_name: str, student_id: str, score: float, runtime: float, timestamp: Optional[str] = None) -> bool:
        try:
            if timestamp is None:
                timestamp = datetime.now().isoformat()
            result_data = {
                "member": member_name,
                "student_id": student_id,
                "score": round(score, 4),
                "runtime": round(runtime, 4),
                "timestamp": timestamp
            }
            filename = f"{member_name.lower()}_results.json"
            filepath = self.results_dir / filename
            with open(filepath, 'w', encoding='utf-8') as f:
                json.dump(result_data, f, indent=2, ensure_ascii=False)
            print(f"✓ Result saved for {member_name}: Score={score:.3f}, Runtime={runtime:.3f}s")
            return True
        except Exception as e:
            print(f"✗ Failed to save result for {member_name}: {e}")
            return False

result_logger = ResultLogger()
print("ResultLogger initialized for Member3")


In [1]:
import cv2
from pyzbar.pyzbar import decode
import os
import json
import numpy as np
from datetime import datetime
import hashlib
import time
from typing import Tuple, List, Optional, Dict, Any
import logging
from dataclasses import dataclass
from pathlib import Path
import customtkinter as ctk
from PIL import Image, ImageTk
from tkinter import messagebox
from deepface import DeepFace

@dataclass
class Config:
    FACE_DB_PATH: str = "student_faces"
    LOG_FILE: str = "verification_log.json"
    STUDENT_DATA_FILE: str = "student_database.json"
    SUPPORTED_EXTENSIONS: List[str] = None

    # Camera settings
    CAMERA_WIDTH: int = 640
    CAMERA_HEIGHT: int = 480
    CAMERA_FPS: int = 25
    
    # Face recognition settings
    SIFT_FEATURES: int = 2000
    SIFT_CONTRAST_THRESHOLD: float = 0.02
    SIFT_EDGE_THRESHOLD: int = 8
    
    # Verification thresholds
    MATCH_THRESHOLD: float = 0.45
    MIN_KEYPOINTS: int = 15
    MIN_GOOD_MATCHES: int = 8
    REQUIRED_CONSECUTIVE: int = 3
    LOWE_RATIO: float = 0.75
    
    # Quality thresholds
    MIN_FACE_SIZE: int = 80
    PREFERRED_FACE_SIZE: int = 120
    MIN_PHOTOS_REQUIRED: int = 5
    DEFAULT_PHOTOS_COUNT: int = 12
    
    def __post_init__(self):
        if self.SUPPORTED_EXTENSIONS is None:
            self.SUPPORTED_EXTENSIONS = ['.jpg', '.jpeg', '.png', '.bmp']


# Setup logging for the backend
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[logging.FileHandler('system.log'), logging.StreamHandler()]
)
logger = logging.getLogger(__name__)


In [2]:
def capture_qr_from_webcam() -> str:
    """Capture and decode a QR code from the webcam feed."""
    print("📱 Starting webcam to capture QR code...")
    cap = cv2.VideoCapture(0)
    detected_qr = None

    while True:
        ret, frame = cap.read()
        if not ret:
            print("⚠️ Failed to read from webcam.")
            continue

        # Decode QR code from frame
        decoded_objects = decode(frame)
        for obj in decoded_objects:
            qr_data = obj.data.decode("utf-8")
            if qr_data:
                print(f"✅ QR Code Detected: {qr_data}")
                detected_qr = qr_data.strip()  # Extract student ID
                # Draw the QR code rectangle
                points = obj.polygon
                hull = cv2.convexHull(np.array([point for point in points], dtype=np.float32))
                cv2.polylines(frame, [np.int32(hull)], True, (0, 255, 0), 3)
                break

        # Display the frame with QR code detection
        cv2.imshow("QR Code Scan", frame)
        
        # Break the loop if QR is detected or the user presses 'q'
        if detected_qr or cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

    if not detected_qr:
        raise Exception("❌ No QR code detected.")
    return detected_qr

def capture_face_from_webcam(output_path="captured_face.jpg"):
    """Capture student's face from the webcam using MTCNN."""
    print("🎥 Starting webcam with MTCNN face detection...")
    cap = cv2.VideoCapture(0)
    captured = False

    while True:
        ret, frame = cap.read()
        if not ret:
            print("⚠️ Failed to read from webcam.")
            continue

        try:
            detections = DeepFace.extract_faces(img_path=frame, detector_backend="mtcnn", enforce_detection=False)
        except Exception as e:
            print(f"⚠️ Detection error: {e}")
            continue

        if detections:
            for face in detections:
                region = face['facial_area']
                (x, y, w, h) = region['x'], region['y'], region['w'], region['h']
                cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
                face_img = frame[y:y+h, x:x+w]
                cv2.imshow("Webcam - Face Detected! Press 's' to save, 'q' to quit", frame)
                key = cv2.waitKey(1) & 0xFF
                if key == ord('s'):
                    cv2.imwrite(output_path, face_img)
                    print(f"✅ Face captured and saved as {output_path}")
                    captured = True
                    break
                elif key == ord('q'):
                    print("❌ Capture cancelled by user.")
                    cap.release()
                    cv2.destroyAllWindows()
                    raise Exception("Webcam capture cancelled.")
        else:
            cv2.imshow("Webcam - No face detected. Press 'q' to quit", frame)
            if cv2.waitKey(1) & 0xFF == ord('q'):
                print("❌ Capture cancelled by user.")
                cap.release()
                cv2.destroyAllWindows()
                raise Exception("Webcam capture cancelled.")

        if captured:
            cv2.waitKey(1000)  # Show the captured frame for 1 second
            break

    cap.release()
    cv2.destroyAllWindows()
    if not captured:
        raise Exception("❌ No face captured from webcam.")

def verify_face(reference_img_path: str, captured_face_path: str) -> float:
    """Verify captured face with reference image using DeepFace."""
    print("🔎 Verifying face with DeepFace...")
    try:
        result = DeepFace.verify(
            img1_path=reference_img_path,
            img2_path=captured_face_path,
            model_name="VGG-Face",  # Using VGG-Face model
            detector_backend="mtcnn",
            enforce_detection=True
        )
        distance = result['distance']
        match_percentage = calculate_match_percentage(distance)
        print(f"📏 Distance: {distance:.4f} | Match Percentage: {match_percentage:.2f}%")
        return match_percentage
    except Exception as e:
        print(f"⚠️ Error verifying face: {e}")
        return 0.0

def calculate_match_percentage(distance: float) -> float:
    """Convert distance to match percentage, ensuring it stays between 0 and 100."""
    if distance < 0.3:
        return 100.0  # Close match, perfect match
    elif distance < 0.5:
        # Adjust the formula to ensure it doesn't exceed 100%
        return max(0.0, min((1 - distance) * 200.0, 100.0))  # Capping at 100%
    else:
        return max(0.0, (1 - distance) * 100.0)  # For distances above 0.5, max at 100%

In [ ]:
class StudentSystem:
    def __init__(self, config: Config = None):
        self.config = config or Config()
        self._setup_directories()
        self.load_student_data()
        self.sift = self._create_sift_detector()
        self.matcher = cv2.BFMatcher(cv2.NORM_L2, crossCheck=False)
        self.face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")
        self.verification_stats = {
            'total_attempts': 0, 'successful_verifications': 0,
            'failed_verifications': 0, 'avg_verification_time': 0.0
        }
        logger.info("Student Verification System backend initialized")

    def _setup_directories(self):
        Path(self.config.FACE_DB_PATH).mkdir(exist_ok=True)

    def _create_sift_detector(self):
        return cv2.SIFT_create(
            nfeatures=self.config.SIFT_FEATURES,
            contrastThreshold=self.config.SIFT_CONTRAST_THRESHOLD,
            edgeThreshold=self.config.SIFT_EDGE_THRESHOLD
        )

    def load_student_data(self):
        try:
            if Path(self.config.STUDENT_DATA_FILE).exists():
                with open(self.config.STUDENT_DATA_FILE, 'r', encoding='utf-8') as f:
                    self.student_data = json.load(f)
                logger.info(f"Loaded data for {len(self.student_data)} students")
            else:
                self.student_data = {}
                logger.info("No existing student data found, starting fresh")
        except (json.JSONDecodeError, IOError) as e:
            logger.error(f"Error loading student data: {e}")
            self.student_data = {}

    def get_student_info(self, student_id: str) -> Optional[Dict[str, Any]]:
        return self.student_data.get(student_id)

    def load_registered_faces(self, student_id: str) -> List[np.ndarray]:
        folder = Path(self.config.FACE_DB_PATH) / student_id
        if not folder.is_dir():
            logger.info(f"No face folder found for student {student_id}")
            return []
        images = []
        for file_path in folder.glob("*"):
            if file_path.suffix.lower() in self.config.SUPPORTED_EXTENSIONS:
                try:
                    img = cv2.imread(str(file_path))
                    if img is not None and img.size > 0:
                        images.append(img)
                        logger.info(f"Loaded image for student {student_id}: {file_path}")
                    else:
                        logger.warning(f"Image file is empty or unreadable: {file_path}")
                except Exception as e:
                    logger.warning(f"Failed to load image {file_path}: {e}")
        logger.info(f"Total images loaded for student {student_id}: {len(images)}")
        return images


In [4]:
class App(ctk.CTk):
    def __init__(self, student_system: StudentSystem):
        super().__init__()
        self.system = student_system
        self.selected_method = None  # To store selected method
        self.cap = None
        self.verification_active = False
        self.current_student_id = None
        self.current_student_name = None
        self.current_student_program = None
        self.reference_data = []
        self.logs_frame = ctk.CTkFrame(self, corner_radius=0, fg_color="transparent")
        self.logs_frame.grid_columnconfigure(0, weight=1)
        self.logs_frame.grid_rowconfigure(0, weight=1)

        self.quality_frame = ctk.CTkFrame(self, corner_radius=0, fg_color="transparent")
        self.quality_frame.grid_columnconfigure(0, weight=1)

        # Add the full UI layout as per the previous code
        self.setup_ui()

    def setup_ui(self):
        self.geometry("1100x720")

        # --- Sidebar Frame ---
        self.sidebar_frame = ctk.CTkFrame(self, width=200, corner_radius=0)
        self.sidebar_frame.grid(row=0, column=0, rowspan=4, sticky="nsew")
        self.sidebar_frame.grid_rowconfigure(5, weight=1)
        
        self.logo_label = ctk.CTkLabel(self.sidebar_frame, text="Main Menu", font=ctk.CTkFont(size=20, weight="bold"))
        self.logo_label.grid(row=0, column=0, padx=20, pady=(20, 10))
        
        self.verify_button = ctk.CTkButton(self.sidebar_frame, text="Verify Student", command=self.verify_student_frame_event)
        self.verify_button.grid(row=1, column=0, padx=20, pady=10)

        self.logs_button = ctk.CTkButton(self.sidebar_frame, text="View Logs", command=self.view_logs_frame_event)
        self.logs_button.grid(row=2, column=0, padx=20, pady=10)

        self.quality_button = ctk.CTkButton(self.sidebar_frame, text="Test Quality", command=self.test_quality_frame_event)
        self.quality_button.grid(row=3, column=0, padx=20, pady=10)

        self.export_button = ctk.CTkButton(self.sidebar_frame, text="Export Report", command=self.export_report_event)
        self.export_button.grid(row=4, column=0, padx=20, pady=10)

        self.exit_button = ctk.CTkButton(self.sidebar_frame, text="Exit", fg_color="transparent", border_width=2, text_color=("gray10", "#DCE4EE"), command=self.on_closing)
        self.exit_button.grid(row=6, column=0, padx=20, pady=20, sticky="s")

        # --- Verification Frame --- Ensure visibility
        self.verify_frame = ctk.CTkFrame(self, corner_radius=0, fg_color="transparent")
        self.verify_frame.grid_columnconfigure(0, weight=1)
        self.verify_frame.grid_rowconfigure(4, weight=1)
        
        self.camera_label = ctk.CTkLabel(self.verify_frame, text="")
        self.camera_label.grid(row=0, column=0, padx=10, pady=10)

        self.status_label = ctk.CTkLabel(self.verify_frame, text="Welcome! Click 'Start Verification' to begin.", font=ctk.CTkFont(size=16))
        self.status_label.grid(row=1, column=0, padx=10, pady=5)
        
        self.info_label = ctk.CTkLabel(self.verify_frame, text="", font=ctk.CTkFont(size=14))
        self.info_label.grid(row=2, column=0, padx=10, pady=5)
        
        self.start_button = ctk.CTkButton(self.verify_frame, text="Start Verification", command=self.start_verification_process)
        self.start_button.grid(row=3, column=0, padx=10, pady=10)

        # --- Make sure frames are properly shown --- 
        self.select_frame_by_name("verify")
        self.protocol("WM_DELETE_WINDOW", self.on_closing)

    def select_frame_by_name(self, name):
        # Hide all frames
        self.verify_frame.grid_remove()
        self.logs_frame.grid_remove()
        self.quality_frame.grid_remove()

        # Show the selected frame
        if name == "verify":
            self.verify_frame.grid(row=0, column=1, sticky="nsew")
        elif name == "logs":
            self.logs_frame.grid(row=0, column=1, sticky="nsew")
        elif name == "quality":
            self.quality_frame.grid(row=0, column=1, sticky="nsew")

    def verify_student_frame_event(self):
        self.select_frame_by_name("verify")

    def view_logs_frame_event(self):
        self.select_frame_by_name("logs")
        logs = self.view_logs()
        
        # Create a text label or scrolling widget to display logs
        self.logs_frame.grid_rowconfigure(1, weight=1)  # Make sure the logs are scrollable
        self.logs_text = ctk.CTkTextbox(self.logs_frame, width=900, height=500)
        self.logs_text.grid(row=1, column=0, padx=10, pady=10)
        self.logs_text.insert("1.0", logs)  # Insert the logs into the textbox
        self.logs_text.configure(state="disabled")  # Make it non-editable

    def view_logs(self) -> str: # MODIFIED: Returns string for GUI
        if not Path(self.system.config.LOG_FILE).exists():
            return "📝 No verification logs found."
        try:
            with open(self.system.config.LOG_FILE, 'r', encoding='utf-8') as f:
                logs = json.load(f)
            if not logs:
                return "📝 No verification logs found."
            
            header = f"{'Timestamp':<20} {'Student ID':<12} {'Name':<25} {'Status':<10} {'Score':<8} {'Time(s)':<8}\n"
            divider = "=" * 90 + "\n"
            report = "📊 Recent Verification Logs\n" + divider + header + divider
            
            for log in reversed(logs[-50:]): # Show newest logs first
                # --- ROBUSTNESS FIX START ---
                try:
                    # Attempt to format the timestamp
                    timestamp = datetime.fromisoformat(log['timestamp']).strftime("%Y-%m-%d %H:%M:%S")
                except (TypeError, ValueError):
                    # If it fails, use a placeholder and continue
                    timestamp = "INVALID TIMESTAMP"
                # --- ROBUSTNESS FIX END ---
    
                status = "✅ SUCCESS" if log.get('success', False) else "❌ FAILED"
                score = f"{log.get('score', 0):.3f}"
                time_taken = f"{log.get('verification_time', 0):.1f}"
                name = log.get('name', 'N/A')[:24]
                student_id = log.get('student_id', 'N/A')
                
                report += f"{timestamp:<20} {student_id:<12} {name:<25} {status:<10} {score:<8} {time_taken:<8}\n"
    
            total = len(logs)
            successful = sum(1 for log in logs if log.get('success'))
            success_rate = (successful / total * 100) if total > 0 else 0
            report += divider + f"📈 Summary: {successful}/{total} successful ({success_rate:.1f}% success rate)"
            return report
        except Exception as e:
            logger.error(f"Error reading logs: {e}")
            return f"❌ Error reading logs: {e}"


    def test_quality_frame_event(self):
        self.select_frame_by_name("quality")
        # You can add logic to test quality here

    def export_report_event(self):
        report_status = self.export_verification_report()
        
        # Create a pop-up or label to show the report status
        self.status_label.configure(text=report_status, text_color="green" if "successfully" in report_status else "red")

    def export_verification_report(self) -> str: # MODIFIED: Returns status string for GUI
        if not Path(self.system.config.LOG_FILE).exists():
            return "No verification logs found to export."
        try:
            with open(self.system.config.LOG_FILE, 'r', encoding='utf-8') as f: logs = json.load(f)
            if not logs: return "No verification data to export."
            
            timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
            report_file = f"verification_report_{timestamp}.json"
            
            total = len(logs)
            successful = sum(1 for log in logs if log['success'])
            success_rate = (successful / total * 100) if total > 0 else 0
            
            report = {
                'generated_at': datetime.now().isoformat(),
                'summary': {
                    'total_attempts': total,
                    'successful_verifications': successful,
                    'success_rate_percentage': round(success_rate, 2),
                },
                'recent_logs': logs[-50:]
            }
            with open(report_file, 'w', encoding='utf-8') as f:
                json.dump(report, f, indent=2, ensure_ascii=False)
            return f"Report successfully exported to {report_file}"
        except Exception as e:
            logger.error(f"Error generating report: {e}")
            return f"Failed to generate report: {e}"

    def on_closing(self):
        try:
            if self.verification_active:
                self.stop_verification_process()
        except Exception as e:
            logger.error(f"Error during closing: {e}")
        finally:
            self.quit()
            self.destroy()

    def stop_verification_process(self):
        self.verification_active = False
        self.verification_step = "idle"
        
        # If the webcam capture is active, stop it
        if self.cap:
            self.cap.release()  # Release the webcam
            self.cap = None  # Reset the webcam object
            
        # Reset UI state for the next run
        self.start_button.configure(state="normal", text="Start Verification")
        
        # Reset student-specific data
        self.current_student_id = None
        self.reference_data = []
        self.consecutive_matches = 0
        self.best_score = 0.0  # For single match; reset it as per top-N logic
        self.match_scores = []  # Reset the list for match percentages used in top-N matching
        self.top_n = 3  # Reset top-N value (if changed)
        
        # Reset any other variables that track face data or frame count
        self.last_known_face = None
        self.frame_count = 0
        self.status_label.configure(text="Verification stopped.", text_color="red")

    def start_verification_process(self):
        self.status_label.configure(text="Starting verification...", text_color="blue")
        self.info_label.configure(text="Please wait...")
        # Stub for capture_student_data
        self.after(1000, self.capture_student_data)

    def capture_student_data(self):
        try:
            # Step 1: Capture QR code (student ID)
            self.status_label.configure(text="Please show your student QR code to the camera.", text_color="blue")
            self.info_label.configure(text="Waiting for QR code...")
            self.update()
            student_id = capture_qr_from_webcam()
            self.status_label.configure(text=f"QR Detected: {student_id}", text_color="green")
            self.info_label.configure(text="Fetching student info...")
            self.update()

            # Step 2: Get student info
            student_info = self.system.get_student_info(student_id)
            if not student_info:
                self.status_label.configure(text="Student not found!", text_color="red")
                self.info_label.configure(text="Please register first.")
                return

            self.status_label.configure(text=f"Welcome {student_info.get('name', '')}", text_color="green")
            self.info_label.configure(text="Please position your face in front of the camera.")
            self.update()

            # Step 3: Capture face
            face_path = f"temp_face_{student_id}.jpg"
            capture_face_from_webcam(output_path=face_path)

            # Step 4: Load reference images
            registered_faces = self.system.load_registered_faces(student_id)
            if not registered_faces:
                self.status_label.configure(text="No reference face found!", text_color="red")
                self.info_label.configure(text="Please register your face first.")
                return

            # Save all reference faces temporarily and compare
            match_scores = []
            top_n = 3  # Set top-N value for averaging

            # Process each reference face
            for idx, ref_img in enumerate(registered_faces):
                ref_path = f"temp_ref_{student_id}_{idx}.jpg"
                cv2.imwrite(ref_path, ref_img)
                
                # Update the status to show processing of each image
                self.status_label.configure(text=f"Processing image {idx + 1} of {len(registered_faces)}...", text_color="blue")
                self.info_label.configure(text="Matching in progress...")
                self.update()
                
                # Get the match percentage for the current reference image
                match_percent = verify_face(ref_path, face_path)
                match_scores.append(match_percent)
                
                # Clean up temp ref file after each comparison
                if os.path.exists(ref_path):
                    os.remove(ref_path)

            # Sort match scores and select the top-N matches
            top_scores = sorted(match_scores, reverse=True)[:top_n]
            average_top = sum(top_scores) / len(top_scores)

            # Step 5: Show result based on the average of top-N matches
            if average_top >= self.system.config.MATCH_THRESHOLD * 100:
                self.status_label.configure(text=f"Verification Success ({average_top:.2f}%)", text_color="green")
                self.info_label.configure(text="Access granted.")
            else:
                self.status_label.configure(text=f"Verification Failed ({average_top:.2f}%)", text_color="red")
                self.info_label.configure(text="Face does not match.")

            # Cleanup temp captured face file
            if os.path.exists(face_path):
                os.remove(face_path)

        except Exception as e:
            self.status_label.configure(text="Error during verification.", text_color="red")
            self.info_label.configure(text=str(e))
            logger.error(f"Verification error: {e}")
            messagebox.showerror("Verification Error", str(e))
            self.info_label.configure(text="Please try again.")


In [5]:
if __name__ == "__main__":
    try:
        # Initialize your config and student system
        config = Config()
        student_system = StudentSystem(config)
        
        # Create and run the application
        app = App(student_system=student_system)
        
        # Start the main application loop
        app.mainloop()
    except Exception as e:
        print(f"Error in UI: {e}")
    finally:
        # Any necessary cleanup (if something fails during startup)
        print("Application is closing.")


2025-08-17 17:45:50,570 - INFO - Loaded data for 2 students
2025-08-17 17:45:50,631 - INFO - Student Verification System backend initialized


Application is closing.
